In [3]:
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt
import bisect
from pyomo.opt import SolverStatus, TerminationCondition

In [22]:
m1 = ConcreteModel()
m1.y = Var(bounds=(-1, 1))
m1.x1 = Var(bounds=(0, 1))
m1.x2 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x2**2 == m1.y**2)
m1.c2 = Constraint(expr=m1.x2 == (m1.x1)**2)
m1.obj = Objective(expr=-m1.x1, sense=minimize)

In [23]:
def v_1(y):  
    return - np.sqrt(np.abs(y))

In [24]:
delta = 1
add_node_num=0
n_samples=5
tolerance=1e-8


y_nodes = np.linspace(-delta, delta, n_samples).tolist()
new_node = 0


print("  ")
print("  ")
print(add_node_num)

# reset bounds
m1.y.setlb(-delta)
m1.y.setub(delta)

for k in range(add_node_num+1):
    print('add node number ',k)
    
    if new_node not in y_nodes:
        bisect.insort(y_nodes, new_node)


    as_nodes = [v_1(y) for y in y_nodes]

    if hasattr(m1, 'As'):
        model_list[i].del_component('As')
    if hasattr(m1, 'pw'):
        model_list[i].del_component('pw')
    #model_list[i].del_component('y')
    #model_list[i].y = Var(bounds=(-delta, delta))
    m1.As = Var()
    m1.pw = Piecewise(
        m1.As, m1.y,
        pw_pts=y_nodes,
        f_rule=as_nodes,
        pw_constr_type='EQ',
        pw_repn='INC'
    )

    solver = SolverFactory('gurobi')
    solver.options['numerics/feastol'] = tolerance
    solver.options['numerics/dualfeastol'] = tolerance
    solver.options['limits/gap'] = tolerance

    
    '''
    old_expr = model_list[i].obj.expr
    model_list[i].del_component('obj')
    model_list[i].obj = Objective(expr=old_expr - model_list[i].As, sense=minimize)
    '''
    #solver.solve(model_list[i])
    # get the ms value
    original_expr = m1.obj.expr
    m1.obj.deactivate()   
    #model_list[i].del_component('obj')
    m1.obj2 = Objective(expr=original_expr - m1.As, sense=minimize)

    if k == 16:
        print('  ')
        print("16start")
        results = solver.solve(model_list[i],tee = True)
        print("16end")
        print('  ')


    results = solver.solve(m1)
    ms_1 = results.problem.lower_bound
    new_node = value(m1.y)
    print('new node is ',new_node)
    print('ms is ',ms_1)
    print(' ')
    print(' ')
    print(' ')
    print(' ')



  
  
0
add node number  0
ERROR: Solver (gurobi) returned non-zero return code (1)
ERROR: Solver log: Set parameter Username Set parameter LicenseID to value
2689755 Academic license - for non-commercial use only - expires 2026-07-21
Read LP format model from file
/var/folders/_9/rch08jqx0f1fp17rg26k4v700000gn/T/tmpt7rk_0n1.pyomo.lp Reading
time = 0.01 seconds x1: 8 rows, 11 columns, 22 nonzeros Traceback (most recent
call last):
      File "<stdin>", line 4, in <module> File
      "/opt/anaconda3/lib/python3.9/site-
      packages/pyomo/solvers/plugins/solvers/GUROBI_RUN.py", line 101, in
      gurobi_run
        model.setParam(key, value)
      File "src/gurobipy/_model.pyx", line 2004, in
      gurobipy._model.Model.setParam File "src/gurobipy/env.pxi", line 395, in
      gurobipy._core.Env.setParam
    gurobipy._exception.GurobiError: Unknown parameter 'numerics/feastol'


ApplicationError: Solver (gurobi) did not exit normally

In [21]:
m1 = ConcreteModel()
m1.y = Var(bounds=(-1, 1))
m1.x1 = Var(bounds=(0, 1))
m1.x2 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x2**2 == m1.y**2)
m1.c2 = Constraint(expr=m1.x2 == (m1.x1)**2)

y_nodes = np.linspace(-1, 1, 5).tolist()
as_nodes = [v_1(y) for y in y_nodes]
    
m1.As = Var()
m1.pw = Piecewise(
    m1.As, m1.y,
    pw_pts=y_nodes,
    f_rule=as_nodes,
    pw_constr_type='EQ',
    pw_repn='INC'
)
m1.obj2 = Objective(expr=-m1.x1 - m1.As, sense=minimize)

#solver = SolverFactory('scip', executable='/Users/yinhuang/miniconda3/bin/scip')
solver = SolverFactory('gurobi')
results = solver.solve(m1)
print(results)


Problem: 
- Name: x1
  Lower bound: -0.17678943333361818
  Upper bound: -0.17677899799800684
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 11
  Number of binary variables: 3
  Number of integer variables: 3
  Number of continuous variables: 8
  Number of nonzeros: 22
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.04070711135864258
  Error rc: 0
  Time: 0.2385859489440918
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

